In [1]:
import sys
import pandas as pd
import csv
import numpy as np
import pickle
sys.path.append("../../code")


from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser





import plotly.plotly as py
import plotly.graph_objs as go


# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
# Connect to database
from MongoConnection import MongoConnection

occ_settings = {
    "host": "chicago.chem-eng.northwestern.edu",
    "port": "27017",
    "db": "plos_citation_position",
    "collection": "occurrences",
    "user": "mongoreader",
    "password": "emptycoffeecup"
}

occ_con = MongoConnection(occ_settings)

In [4]:
def write_mongodb_to_file(db, fname, query={}):
    """ Write a mongodb to a csv file. Due to memory issues, this looping method was chosen rather than converting to dataframe
    immediately. Can potentially work with just the csv, but pickle is assumed for most other notebooks. HDF would be
    better still, but had issues with assembly"""

    size_db = occ_con.collection.count()
    
    rows_done = 0
    batch_size = 100000 # number of rows to retrieve, can bump up on better computers
    
    # If connection is interrupted, can change this start bound to the last lower_bound printed. 
    # Also turn header_flag off if continuing in this way
    lower_bound = 0 
    header_flag = True
    
    while lower_bound < size_db:
        
        cursor = db.collection.find(query).limit(batch_size).skip(lower_bound) # query database
        
        df =  pd.DataFrame(list(cursor)) 
        #df['paper_UT'] = df['paper_UT'].astype(str) # After solving most errors, this should be unnecessary. Turn back on if something goes wrong with this column
        
        del df['_id']
        print('----------')
        df.to_csv(fname, mode='a', sep='\t', header=header_flag, encoding='utf-8')
        lower_bound += batch_size
        header_flag = False # Turn off headers after first one
        print(lower_bound)

    return lower_bound

In [2]:
write_mongodb_to_file(occ_con, '../../data/itcs.csv', query={})

NameError: name 'write_mongodb_to_file' is not defined

In [ ]:
# If there's some interruption, you can check where the file stopped appending

#input_file = open("../../data/itcs.csv","r+")
#reader_file = csv.reader(input_file)
#value = len(list(reader_file))

In [3]:
# Had some issues with strings and such occuring in certain numeric columns, this function is passed to the converter option of pd.read_csv
# Columns like the first one, 'cite_count' also could be just as easily represented as int64, but again, some conversion issues. In this case, it's not a huge problem
def to_npint(val):
    return np.int64(val)

In [4]:
# Read as csv - ignore the first column of dummy indexing. Remember to change non-robust usecols range value if new columns are added to mongo database
df = pd.read_csv('../../data/itcs.csv', sep='\t', usecols=range(40)[1:], converters= {'paper_cite_count': to_npint})

In [5]:
df.shape

(10848620, 39)

## Now some preprocessing can be done here as well. Primary two factors are regex section-assignment and datetime processing, these are added as new columns

## Both functions may be altered in the code

In [6]:
# Here's where we might run into issues. With the conversion to mongo and then back to csv and into dataframe, there
# may be some issues with previous functions. The main culprit is the multiple NaN types. To get around this, for
# string columns with NaNs, I'm converting the NaNs to empty strings, which allow the string methods needs to do
# preprocessing and analysis
nan_cols = df.columns[df.dtypes=='object']
df[nan_cols]=df[nan_cols].fillna('')

In [8]:
# Preprocessing for section-mapping with regular expressions as well as conversion from numerics and text to datetime
# for publication date takes times, so it's better to have it done here.

# Note that section_regex_parser can be changed depending on target section ids, while datetime_filler can be changed
# to use different assumptions for date assignment when missing specific day fields. Seed for random set for continuity
from itc_preprocessing import section_regex_parser, datetime_filler
import random
random.seed(111)

In [9]:
# Here we use the parser on the sect_title and sect_title_altcolumns in the df. This gives a list of string matches. 
# This is remapped to indices, and then added to the new column
judgement = section_regex_parser(df)
judgement_2 = []
sect_index_dict = {'intro': 0, 'methods': 1, 'results': 2, 'disc': 3, 'res_disc':4, 'concl':5, 'mixed':6, 'na':7}
for i in judgement:
    judgement_2.append(sect_index_dict[i])
df['regex_sect_index'] = judgement_2

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000


In [10]:
# Then, we make datetime columns for both PLOS and reference papers.

dt_ = datetime_filler(df, 'ref_pub_date', 'ref_pub_year')
dt_plos_ = datetime_filler(df, 'plos_pub_date', 'plos_pub_year')

df['ref_datetime'] = dt_
df['plos_datetime'] = dt_plos_

['0']
    0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
17000

## Depending on your preference, you can work with the csv straight away, though previous
## function will have issues.
## Since programs already have a pickle load setup, here's a couple options.

## Split and save to 3 pickle files, or save full pickle, your choice. 

### Option 1: Split

In [12]:
# Now to make our two smaller dataframes with the new data, simply select the associated colummns.
# (The ref_dataframe_min df doesn't include any additional info gathered this way, so it's just these two)

plos_df = df[['paper_UT','paper_cite_count','total_refs', 'paper_char_total', 'paper_word_total', 'plos_pub_date', 'plos_pub_year', 'plos_article_type', 'plos_field', 'plos_j1', 'plos_j2', 'plos_j9', 'plos_ji']]
citation_df = df[['reference_UT','cite_count','ref_pub_date','ref_pub_year', 'ref_article_type', 'ref_field', 'ref_j1', 'ref_j2', 'ref_j9', 'ref_ji']]

# Drop all non-unique fields. Note that coincidentally overlapping rows with -1 for ut will be dropped as well
plos_df_unique = plos_df.drop_duplicates('paper_UT')
citation_df_unique = citation_df.drop_duplicates('reference_UT')



# For completeness, then drop all non-UT fields that are already contained in the plos_df and citation_df dataframes
ref_df_min = df.drop(['paper_cite_count','total_refs', 'paper_char_total', 'paper_word_total', 'plos_pub_date', 'plos_pub_year', 'plos_article_type', 'cite_count','ref_pub_date','ref_pub_year', 'ref_article_type', 'plos_field','ref_field', 'plos_field', 'plos_j1', 'ref_j1', 'ref_j2',
       'ref_j9', 'ref_ji', 'plos_j2', 'plos_j9', 'plos_ji'], 1)

In [13]:
del df # to avoid memory issues when writing, i delete the original df

In [14]:
# Reassign the index to be the respective UT column. May have to manually drop non-information rows with 
#(indicated by a -1 UT, as no information is possible to be matched in primary dataframe)

plos_df_small = plos_df_unique.set_index('paper_UT')
#plos_df_small = plos_df_small.drop(-1,0)
citation_df_small = citation_df_unique.set_index('reference_UT')
#citation_df_small = citation_df_small.drop(-1,0)

In [15]:
#Lastly, pickle them
with open('../../data/plos_paper_dataframe.pkl', 'wb') as handle:
    pickle.dump(plos_df_small, handle, protocol = 2)
print ("written: ../../data/plos_paper_dataframe.pkl")

with open('../../data/citation_dataframe.pkl', 'wb') as handle:
    pickle.dump(citation_df_small, handle, protocol = 2)
print ("written: ../../data/citation_dataframe.pkl")    

with open('../../data/ref_dataframe_min.pkl', 'wb') as handle:
    pickle.dump(ref_df_min, handle, protocol = 2)
print ("written: ../../data/ref_dataframe_min.pkl")

written: ../../data/plos_paper_dataframe.pkl
written: ../../data/citation_dataframe.pkl
written: ../../data/ref_dataframe_min.pkl


In [16]:
plos_df_small

,paper_cite_count,total_refs,paper_char_total,paper_word_total,plos_pub_date,plos_pub_year,plos_article_type,plos_field,plos_j1,plos_j2,plos_j9,plos_ji
paper_UT,,,,,,,,,,,,
324515600133,2,37,21733,3780,SEP 4,2013.0,@ Article,D RO MULTIDISCIPLINARY SCIENCES,PLOS ONE,PLOS ONE,PLOS ONE,PLoS One
352084800063,2,41,29908,5312,MAR 31,2015.0,@ Article,['D RO MULTIDISCIPLINARY SCIENCES'],PLOS ONE,PLOS ONE,PLOS ONE,PLoS One
322590800016,19,86,41379,7201,JUL,2013.0,R Review,"['D PY MEDICINE, GENERAL & INTERNAL']",PLOS MED,PLOS MED,PLOS MED,PLos Med.
358595900105,1,17,15208,2615,JUL 28,2015.0,@ Article,['D RO MULTIDISCIPLINARY SCIENCES'],PLOS ONE,PLOS ONE,PLOS ONE,PLoS One
312910200007,8,33,16838,3033,DEC,2012.0,@ Article,"['D TI PARASITOLOGY', 'D YU TROPICAL MEDICINE']",PLO NE TR D,PLOS NEGLECT TROP D,PLOS NEGLECTED TROP DIS,Plos Neglect. Trop. Dis.
254928800015,98,47,21778,3839,FEB,2008.0,@ Article,"['D PY MEDICINE, GENERAL & INTERNAL']",PLOS MED,PLOS MED,PLOS MED,PLos Med.
307437900020,25,53,39062,6657,AUG 7,2012.0,@ Article,"['D CU BIOLOGY', 'D RO MULTIDISCIPLINARY SCIEN...",PLOS ONE,PLOS ONE,PLOS ONE,PLoS One
271022300001,1,34,32249,6027,OCT 23,2009.0,@ Article,D CU BIOLOGY,PLOS ONE,PLOS ONE,PLOS ONE,PLoS One
288809100001,12,65,25717,4406,MAR 22,2011.0,@ Article,D CU BIOLOGY,PLOS ONE,PLOS ONE,PLOS ONE,PLoS One


In [17]:
citation_df_small


,cite_count,ref_pub_date,ref_pub_year,ref_article_type,ref_field,ref_j1,ref_j2,ref_j9,ref_ji
reference_UT,,,,,,,,,
A1995QY75100004,60.0,MAY,1995.0,@ Article,D ZA UROLOGY & NEPHROLOGY,AM J KIDNEY,AM J KIDNEY DIS,AMER J KIDNEY DIS,Am. J. Kidney Dis.
000222499800017,65.0,JUN,2004.0,@ Article,D ZA UROLOGY & NEPHROLOGY,AM J KIDNEY,AM J KIDNEY DIS,AMER J KIDNEY DIS,Am. J. Kidney Dis.
000263911400006,5.0,APR,2009.0,@ Article,"['D YP TRANSPLANTATION', 'D YA SURGERY']",TRANSPLAN I,TRANSPL INT,TRANSPLANT INT,Transpl. Int.
000289279600018,29.0,MAY,2011.0,@ Article,['D ZA UROLOGY & NEPHROLOGY'],J UROL,J UROLOGY,J UROL,J. Urol.
000287717700023,19.0,,2011.0,@ Article,['D ZA UROLOGY & NEPHROLOGY'],CONTR NEPHR,CONTRIB NEPHROL,CONTRIB NEPHROL,Contrib.Nephrol.
-1,NaN,,NaN,,,,,,
000257589600013,27.0,AUG,2008.0,R Review,D ZA UROLOGY & NEPHROLOGY,J UROL,J UROLOGY,J UROL,J. Urol.
000246465200001,29.0,APR,2007.0,R Review,D DR CELL BIOLOGY,BIOCELL,BIOCELL,BIOCELL,Biocell
000238112100022,97.0,JUN,2006.0,@ Article,['D ZA UROLOGY & NEPHROLOGY'],KIDNEY INT,KIDNEY INT,KIDNEY INT,Kidney Int.


In [22]:
ref_df_min

,occurence,paper_UT,paper_char_pos,paper_word_pos,para_char_pos,para_char_total,para_index,para_word_pos,para_word_total,reference_UT,...,sect_char_pos,sect_char_total,sect_index,sect_word_pos,sect_word_total,section_title,section_title_alt,regex_sect_index,ref_datetime,plos_datetime
0,1,324515600133,139,21,139,820,0,21,130,A1995QY75100004,...,139,4029,0,21,658,Introduction,introduction,0,1995-05-15 00:00:00,2013-09-04 00:00:00
1,2,324515600133,16829,2992,494,1208,24,76,188,A1995QY75100004,...,494,5398,3,76,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
2,3,324515600133,17257,3057,922,1208,24,141,188,A1995QY75100004,...,922,5398,3,141,864,Discussion,discussion,3,1995-05-15 00:00:00,2013-09-04 00:00:00
3,1,324515600133,266,42,266,820,0,42,130,000222499800017,...,266,4029,0,42,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00
4,2,324515600133,444,73,444,820,0,73,130,000222499800017,...,444,4029,0,73,658,Introduction,introduction,0,2004-06-15 00:00:00,2013-09-04 00:00:00
5,3,324515600133,16830,2993,495,1208,24,77,188,000222499800017,...,495,5398,3,77,864,Discussion,discussion,3,2004-06-15 00:00:00,2013-09-04 00:00:00
6,4,324515600133,17110,3035,775,1208,24,119,188,000222499800017,...,775,5398,3,119,864,Discussion,discussion,3,2004-06-15 00:00:00,2013-09-04 00:00:00
7,5,324515600133,17258,3058,923,1208,24,142,188,000222499800017,...,923,5398,3,142,864,Discussion,discussion,3,2004-06-15 00:00:00,2013-09-04 00:00:00
8,1,324515600133,142,22,142,820,0,22,130,000263911400006,...,142,4029,0,22,658,Introduction,introduction,0,2009-04-15 00:00:00,2013-09-04 00:00:00
9,2,324515600133,445,74,445,820,0,74,130,000263911400006,...,445,4029,0,74,658,Introduction,introduction,0,2009-04-15 00:00:00,2013-09-04 00:00:00


### Option 2: Single pickle file (not preferred)

In [ ]:
with open('../../data/itc_df.pkl', 'wb') as handle:
    pickle.dump(df, handle, protocol = 2)

In [ ]:
# Extra checks below, mainly checking for column type homogeneity

In [ ]:
type_list_2 = []
for i in df['section_title_alt']:
    if isinstance(i, float):
        if i not in type_list_2:
            type_list_2.append(i)
print(type_list_2)

In [ ]:
cols = nan_cols
for j in cols:
    type_list =[]
    for i in df[j]:
        if type(i) not in type_list:
            type_list.append(type(i))
            print(i)
    print(type_list)

My own selection of fields

In [ ]:
#sorted(df.columns)

# ['cite_count',
#  'occurence',
#  'paper_UT',
#  'paper_char_pos',
#  'paper_char_total',
#  'paper_cite_count',
#  'paper_word_pos',
#  'paper_word_total',
#  'para_char_pos',
#  'para_char_total',
#  'para_index',
#  'para_word_pos',
#  'para_word_total',
#  'plos_article_type',
#  'plos_field',
#  'plos_j1',
#  'plos_j2',
#  'plos_j9',
#  'plos_ji',
#  'plos_pub_date',
#  'plos_pub_year',
#  'ref_article_type',
#  'ref_field',
#  'ref_j1',
#  'ref_j2',
#  'ref_j9',
#  'ref_ji',
#  'ref_pub_date',
#  'ref_pub_year',
#  'reference_UT',
#  'reference_rank',
#  'sect_char_pos',
#  'sect_char_total',
#  'sect_index',
#  'sect_word_pos',
#  'sect_word_total',
#  'section_title',
#  'section_title_alt',
#  'total_refs']



In [ ]:
# Now to make a smaller dataframe with the new data, simply select the associated colummns.
selection_df = df[['section_simplified','ref_pub_year',  'reference_UT', 'paper_UT','paper_cite_count', 'reference_rank','ref_field',  'ref_j1','total_refs', 'plos_pub_year', 'plos_field', 'cite_count', 'occurence'  ]]

print (selection_df.shape)
# selection_df_unique = selection_df.drop_duplicates()
# print (selection_df_unique.shape)




# Drop all non-unique fields. Note that coincidentally overlapping rows with -1 for ut will be dropped as well
# plos_df_unique = plos_df.drop_duplicates('paper_UT')


# For completeness, then drop all non-UT fields that are already contained in the plos_df and citation_df dataframes
# ref_df_min = df.drop(['paper_cite_count','total_refs', 'paper_char_total', 'paper_word_total', 'plos_pub_date', 'plos_pub_year', 'plos_article_type', 'cite_count','ref_pub_date','ref_pub_year', 'ref_article_type', 'plos_field','ref_field', 'plos_field', 'plos_j1', 'ref_j1', 'ref_j2',
#        'ref_j9', 'ref_ji', 'plos_j2', 'plos_j9', 'plos_ji'], 1)






# Reassign the index to be the respective UT column. May have to manually drop non-information rows with 
#(indicated by a -1 UT, as no information is possible to be matched in primary dataframe)

# plos_df_small = plos_df_unique.set_index('paper_UT')
# #plos_df_small = plos_df_small.drop(-1,0)
# citation_df_small = citation_df_unique.set_index('reference_UT')
# #citation_df_small = citation_df_small.drop(-1,0)




#Lastly, pickle them
filename='../../data/selection_df.pickle'
with open(filename, 'wb') as handle:
    pickle.dump(selection_df, handle, protocol = 2)
print ("written:", filename)

In [ ]:

df_clean  = selection_df[selection_df["paper_UT"] == -1]

In [ ]:
df_clean.shape

In [ ]:
selection_df


In [ ]:
df.groupby(["section_title_alt"]).size()

In [ ]:
df.columns